In [1]:
from neural_network import LossFunctions
from neural_network import datasets
from neural_network.classes.Optimizers import NesterovSGD
from neural_network.classes.Validation import ConfigurationGenerator, Hyperparameter, model_builder
import neural_network.utils
import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale

In [2]:
hp = Hyperparameter(
    generator_logic="random_choice_from_range",
    generator_space= (3.0, 5),
    unfold=False
)

next(hp)

4.920817942947524

# Configure the hyperparameter grid

In [6]:
generator = ConfigurationGenerator(
	mode = "random",
	num_trials = 3,
	folded_hyper_space={
		
		"loss_function" : Hyperparameter(
				generator_logic="all_from_list",
				generator_space=[ LossFunctions.MSE ]
		),
		"optimizer" : Hyperparameter(
				generator_logic="all_from_list",
				generator_space=[ NesterovSGD ]
		),
		"optimizer_learning_rate" : Hyperparameter(
				generator_logic="all_from_list",
				generator_space=[ 0.1 ]
		),
		"optimizer_momentum" : Hyperparameter(
				generator_logic="random_choice_from_range",
				generator_space= (0.7, 0.9 )
		),
		"optimizer_regularization" : Hyperparameter(
				generator_logic="all_from_list",
				generator_space=[ 0. ,0.1, 0.2  ]
		),
		"batch_size" : Hyperparameter(
				generator_logic="all_from_list",
				generator_space=[ 200 ]
		),
		"num_epochs" : Hyperparameter(
				generator_logic="all_from_list",
				generator_space=[ 1000 ]
		),
		"num_hidden_layers" : Hyperparameter(
				generator_logic="all_from_list",
				generator_space=[ 3 ]
		),
		"neurons_in_layer_1" : Hyperparameter(
				generator_logic="all_from_list",
				generator_space=[ 30 ]
		),
		"neurons_in_layer_2" : Hyperparameter(
				generator_logic="all_from_list",
				generator_space=[ 30 ]
		),
		"neurons_in_layer_3" : Hyperparameter(
				generator_logic="all_from_list",
				generator_space=[ 30 ]
		)
	}
)

for i in generator:
	print(i, "\n")

{'loss_function': <class 'neural_network.classes.LossFunctions.MSE.MSE'>, 'optimizer': <class 'neural_network.classes.Optimizers.NesterovSGD.NesterovSGD'>, 'optimizer_learning_rate': 0.1, 'optimizer_momentum': 0.7361758810216454, 'optimizer_regularization': 0.0, 'batch_size': 200, 'num_epochs': 1000, 'num_hidden_layers': 3, 'neurons_in_layer_1': 30, 'neurons_in_layer_2': 30, 'neurons_in_layer_3': 30} 

{'loss_function': <class 'neural_network.classes.LossFunctions.MSE.MSE'>, 'optimizer': <class 'neural_network.classes.Optimizers.NesterovSGD.NesterovSGD'>, 'optimizer_learning_rate': 0.1, 'optimizer_momentum': 0.7356305042865241, 'optimizer_regularization': 0.2, 'batch_size': 200, 'num_epochs': 1000, 'num_hidden_layers': 3, 'neurons_in_layer_1': 30, 'neurons_in_layer_2': 30, 'neurons_in_layer_3': 30} 

{'loss_function': <class 'neural_network.classes.LossFunctions.MSE.MSE'>, 'optimizer': <class 'neural_network.classes.Optimizers.NesterovSGD.NesterovSGD'>, 'optimizer_learning_rate': 0.1, 

# Dataset preprocessing

In [4]:
dataset_attribute_columns = ["a1", "a2", "a3", "a4", "a5", "a6"]
dataset_class_column = "class"

tr_df, vl_df, _ = neural_network.utils.split_samples(
	df=datasets.read_monk1()[0],
	tr_size=0.7,
	vl_size=0.3,
	ts_size=0.0
)

tr_df = pd.get_dummies(tr_df, columns=dataset_attribute_columns)
vl_df = pd.get_dummies(vl_df, columns=dataset_attribute_columns)

tr_outputs = tr_df.pop("class").to_numpy()
tr_inputs = tr_df.to_numpy(dtype=np.float32)
vl_outputs = vl_df.pop("class").to_numpy()
vl_inputs = vl_df.to_numpy(dtype=np.float32)

tr_outputs = minmax_scale(tr_outputs)
tr_inputs = minmax_scale(tr_inputs)
vl_outputs = minmax_scale(vl_outputs)
vl_inputs = minmax_scale(vl_inputs)

In [5]:
for config in generator:
    model = model_builder(
        config=config,
        input_shape= tr_inputs.shape[-1],
        output_shape=1,
        verbose=True
    )

    model.fit(tr_inputs, tr_outputs, validation_data=[vl_inputs, vl_outputs])

model builded


Exception: input shape not matching the shape provided